# 01 — Загрузка данных EUR/USD из OANDA

Загружаем конфиг OANDA, подключаемся к v20 REST API, скачиваем свечи EUR/USD для M15 и H1,
сохраняем данные в data/ и строим простой график для проверки.

In [ ]:
import logging
from datetime import datetime, timezone
from pathlib import Path
import sys

from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import pandas as pd

PROJECT_ROOT = Path.cwd().resolve()
if (PROJECT_ROOT / "src").exists():
    ROOT = PROJECT_ROOT
elif (PROJECT_ROOT.parent / "src").exists():
    ROOT = PROJECT_ROOT.parent
else:
    ROOT = PROJECT_ROOT

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.oanda_client import OandaClient, OandaConfig

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

DATA_DIR = ROOT / "data"
CONFIG_PATH = ROOT / "config" / "oanda_config.json"
if not CONFIG_PATH.exists():
    CONFIG_PATH = ROOT / "config" / "oanda_config.example.json"

DATA_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
config = OandaConfig.from_json(CONFIG_PATH)
client = OandaClient(config=config)

print(f"Base URL: {config.base_url}")


In [ ]:
account_details = client.get_account_details()
account_details


По умолчанию скачиваем ~24 месяца истории. При необходимости можно переключиться на быстрый
режим с ограниченным количеством свечей.

In [ ]:
INSTRUMENT = "EUR_USD"
USE_RANGE = True

if USE_RANGE:
    end_dt = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)
    start_dt = end_dt - relativedelta(months=24)

    eurusd_m15 = client.get_candles_range(INSTRUMENT, "M15", start=start_dt, end=end_dt)
    eurusd_h1 = client.get_candles_range(INSTRUMENT, "H1", start=start_dt, end=end_dt)
else:
    eurusd_m15 = client.get_candles(INSTRUMENT, "M15", count=5000)
    eurusd_h1 = client.get_candles(INSTRUMENT, "H1", count=5000)


In [ ]:
print("M15:", eurusd_m15.shape, eurusd_m15["time"].min(), eurusd_m15["time"].max())
print("H1 :", eurusd_h1.shape, eurusd_h1["time"].min(), eurusd_h1["time"].max())
print("M15 duplicates:", eurusd_m15["time"].duplicated().sum())
print("H1 duplicates :", eurusd_h1["time"].duplicated().sum())


In [ ]:
plt.figure(figsize=(12, 4))
eurusd_m15.tail(2000).plot(x="time", y="close", ax=plt.gca(), label="EUR/USD M15")
plt.title("EUR/USD M15 Close (last 2000)")
plt.xlabel("Time")
plt.ylabel("Close")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
m15_path = DATA_DIR / "eurusd_M15.parquet"
h1_path = DATA_DIR / "eurusd_H1.parquet"

eurusd_m15.to_parquet(m15_path, index=False)
eurusd_h1.to_parquet(h1_path, index=False)

print(f"Saved M15 to: {m15_path}")
print(f"Saved H1 to: {h1_path}")
